# Process data
This notebook uses the sanitized data from `download-and-sanitize`

In [80]:
import os
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def store_csv(df, filename, folder):
    print(f"Storing {filename} to output folder")
    fullpath = os.path.join(folder, filename)
    df.to_csv(fullpath, index=False, sep =',', decimal='.', float_format='%.5f')

def get_or_create_folder(baseDir, folderName):
    path = os.path.join(baseDir, folderName)
    if(not os.path.exists(path)):
        os.mkdir(path)
    return path

current_dir = os.getcwd()

sanitized_folder = get_or_create_folder(current_dir, 'sanitized')
processed_folder = get_or_create_folder(current_dir, 'processed')
csv_yearly = get_or_create_folder(sanitized_folder, 'cnsfails-yearly')


In [81]:
def read_csv(folder, name):
    li = []
    print(f"Reading all files in memory for pandas, this will take some time")
    for path in Path(folder).rglob(f'*{name}*.csv'):
        filename = os.path.basename(path)
        print(f"Read file {filename}")
        csv_df = pd.read_csv(path, index_col=None, header=0, sep =',', decimal='.', parse_dates=['DATE'],\
            dtype={'SYMBOL': 'str', 'QUANTITY': 'int', 'PRICE': 'float'})
        li.append(csv_df)
    
    print(f"Fles read, generating dataframe")
    df = pd.concat(li, axis=0, ignore_index=True)
    print(f"dataframe generated")
    return df


# You can easily also just filter for one year: e.g. 'cnsfails-2021'
df = read_csv(csv_yearly, 'cns')
df.count()

Reading all files in memory for pandas, this will take some time
Read file cnsfails-2004.csv
Read file cnsfails-2005.csv
Read file cnsfails-2006.csv
Read file cnsfails-2007.csv
Read file cnsfails-2008.csv
Read file cnsfails-2009.csv
Read file cnsfails-2010.csv
Read file cnsfails-2011.csv
Read file cnsfails-2012.csv
Read file cnsfails-2013.csv
Read file cnsfails-2014.csv
Read file cnsfails-2015.csv
Read file cnsfails-2016.csv
Read file cnsfails-2017.csv
Read file cnsfails-2018.csv
Read file cnsfails-2019.csv
Read file cnsfails-2020.csv
Read file cnsfails-2021.csv
Fles read, generating dataframe
dataframe generated


DATE      21348881
SYMBOL    21348871
FAILS     21348881
PRICE     21348881
dtype: int64

In [82]:
# Fail value in million
df['FAILVALUE'] = (df['FAILS'] * df['PRICE']) / 1000000
df = df.sort_values(by=['FAILVALUE'], ascending=False)
df.head()

,DATE,SYMBOL,FAILS,PRICE,FAILVALUE
2159998,2007-06-27,IWM,134504697,81.74,10994.413933
2156632,2007-06-26,IWM,126547156,82.49,10438.874898
2153448,2007-06-25,IWM,95656140,82.92,7931.807129
20826571,2021-06-21,SPY,14271236,414.92,5921.421241
7879982,2011-08-15,SPY,49563253,118.15,5855.898342


In [83]:
df_symbols = df.groupby('SYMBOL').agg({'DATE':'count', 'FAILS': 'sum', 'FAILVALUE': 'sum'}).reset_index()
df_symbols = df_symbols.rename(columns={"DATE": "DAYCOUNT"})
df_symbols = df_symbols.sort_values(by=['FAILVALUE', 'FAILS'], ascending=False)

store_csv(df_symbols, f"cnsfails-by-symbol.csv", processed_folder)

Storing cnsfails-by-symbol.csv to output folder


In [84]:
df_days = df.groupby('DATE').agg({'FAILS': 'sum', 'FAILVALUE': 'sum'}).reset_index()
df_days = df_days.sort_values(by=['DATE'])

store_csv(df_days, f"cnsfails-by-day.csv", processed_folder)

Storing cnsfails-by-day.csv to output folder


In [90]:
df_percentage = df.merge(df_days, how='left', on='DATE', suffixes=('', '_TOTAL'))
df_percentage['FAILS_PERCENT'] = (df_percentage['FAILS'] / df_percentage['FAILS_TOTAL'])
df_percentage['FAILVALUE_PERCENT'] = (df_percentage['FAILVALUE'] / df_percentage['FAILVALUE_TOTAL'])
df_percentage = df_percentage.sort_values(by=['FAILVALUE_PERCENT'], ascending=False)
df_percentage = df_percentage.drop(columns=['FAILS_TOTAL', 'FAILVALUE_TOTAL'])
df_percentage.head()

,DATE,SYMBOL,FAILS,PRICE,FAILVALUE,FAILS_PERCENT,FAILVALUE_PERCENT
5,2015-09-15,SPY,28432148,196.01,5572.985329,0.143663,0.705634
68,2009-10-14,SPY,20859311,107.50,2242.375933,0.046250,0.675021
18,2015-09-11,SPY,20876457,195.85,4088.654103,0.107392,0.645724
1,2007-06-26,IWM,126547156,82.49,10438.874898,0.121910,0.644688
8,2015-09-14,SPY,25361596,196.74,4989.640397,0.145590,0.643690


In [92]:
def filtered_percentage(df, column, threshold):
    df_filtered = df[df[column] >= threshold]
    df_filtered = df_filtered.sort_values(by=[column], ascending=False)
    store_csv(df_filtered, f"cnsfails-filtered-{column}.csv", processed_folder)
    print(f"{column} >={threshold*100}%: {len(df_filtered.index)}")
    df_filtered.head()

threshold_percent = .05

filtered_percentage(df_percentage, 'FAILVALUE_PERCENT', threshold_percent)
filtered_percentage(df_percentage, 'FAILS_PERCENT', threshold_percent)

Storing cnsfails-filtered-FAILVALUE_PERCENT-percentage.csv to output folder
FAILVALUE_PERCENT >=5.0%: 5793
Storing cnsfails-filtered-FAILS_PERCENT-percentage.csv to output folder
FAILS_PERCENT >=5.0%: 9312


In [87]:
threshold_daily_million = 100

df_small_fails = df[df['FAILVALUE'] < threshold_daily_million]
print(f"Daily failvalue below {threshold_daily_million} million: {len(df_small_fails.index)}")
df_small_merged = df_small_fails.groupby('DATE').agg({'FAILS': 'sum', 'FAILVALUE': 'sum'}).reset_index()
df_small_merged['SYMBOL'] = "Other"
df_small_merged ['PRICE'] = 0.0

df_big_fails = df[df['FAILVALUE'] >= threshold_daily_million]
print(f"Daily failvalue above {threshold_daily_million} million: {len(df_big_fails.index)}")

df_filtered = df_big_fails.append(df_small_merged)
df_filtered = df_filtered.sort_values(by=['DATE', 'FAILVALUE'])
store_csv(df_filtered, f"cnsfails-filtered-failvalue.csv", processed_folder)


Daily failvalue below 100 million: 21339221
Daily failvalue above 100 million: 9660
Storing cnsfails-filtered-failvalue.csv to output folder


In [88]:
def filter_price_range_daily(df, category, min, max):
    df_filtered = df[(df['PRICE'] >= min) & (df['PRICE'] < max)]
    print(f"Price {min}-{max}: {len(df_filtered.index)}")
    df_days = df_filtered.groupby('DATE').agg(FAILS = ('FAILS', 'sum'), FAILVALUE = ('FAILVALUE', 'sum'), ENTRIES = ('SYMBOL', 'count')).reset_index()
    df_days['PRICE_CATEGORY'] = category
    df_days = df_days[['DATE', 'PRICE_CATEGORY', 'ENTRIES', 'FAILS', 'FAILVALUE']]
    return df_days

# Create categories for <$5, $5-$50, $50-$500, >$500
df_price_a = filter_price_range_daily(df, 'A', 0, 5)
df_price_b = filter_price_range_daily(df, 'B', 5, 50)
df_price_c = filter_price_range_daily(df, 'C', 50, 500)
df_price_d = filter_price_range_daily(df, 'D', 500, 500000)

df_price_categories = df_price_a.append(df_price_b).append(df_price_c).append(df_price_d)
df_price_categories = df_price_categories.sort_values(by=['DATE', 'PRICE_CATEGORY'])
store_csv(df_price_categories, 'cnsfails-by-day-price-categories.csv', processed_folder)

Price 0-5: 7318515
Price 5-50: 11289498
Price 50-500: 2706452
Price 500-500000: 34416
Storing cnsfails-by-day-price-categories.csv to output folder
